In [95]:
import requests
import pandas as pd
from datetime import datetime

url = 'https://api.vk.com/method/wall.get'
TOKEN = 'bd609931bd609931bd609931e1bd1be35ebbd60bd609931df400a080084c6e2a88bf067'

def get_last_news(group_name, day, time_start, time_end, url=url, token=TOKEN):

    offset = 0
    feed = {}
    
    print(f'Читаю новости {group_name}...')
    # просматриваем по одной новости в цикле от 18.00 до 09.00
    while True:
        params = {
        'domain':group_name,
        'filter':'all',
        'count':1,
        'offset':offset,
        'v':5.131,
        'access_token':token
        }

        post = requests.get(url, params=params).json()['response']['items'][0]
        
        if post['date'] > time_end:
            offset += 1
            continue
        elif post['date'] < time_start:
            print('Прочитано.')
            break
        
        text = post['text']
        if text:
            date = datetime.utcfromtimestamp(post['date']+3600*7).strftime('%Y-%m-%d %H:%M:%S')
            link = 'https://vk.com/vk?w=wall'+str(post['owner_id'])+'_'+str(post['id'])
            feed[date] = [text, link, group_name]
            
        offset += 1

    return feed

vk_groups = [
    'tomsk123',
    'ghest_tomsk',
    'ya_tomich',
    'tomskie',
    'typical_tomsk',
    'tomskvkurse',
    'inctsk',
    'znaigorod',
    'goodcity70',
    'black_list_tomsk',
    'dortom'
]

day = input('Введите дату в формате ГГГГ-ММ-ДД или нажмите Enter для текущего дня:\n')

# попробуем прочитать необходимую дату, если не получится, берем сегодняшний день
if len(day)>0:
    try:
        today = datetime.strptime(day, '%Y-%m-%d').date()
    except Exception as err:
        print('Ошибка: некорректная дата')
        raise(err)
        
else:
    today = datetime.now()

print(f'Читаю новости за {today}')
# Будем читать новости с 9.00 до 18.00
time_start = datetime(today.year, today.month, today.day, 9, 0).timestamp()
time_end = datetime(today.year, today.month, today.day, 18, 0).timestamp()

try:
    df = {}
    for group in vk_groups:
        df.update(get_last_news(group_name=group, day=today, time_start=time_start, time_end=time_end))

    df = pd.DataFrame(df).T
    df.columns = ['text', 'link','group']
    print('Записываю файл excel...')
    df.to_excel(today.strftime('%Y-%m-%d--%H-%M-%S')+'.xlsx')
    print('Готово, можете пользоваться!')
except Exception as err:
    print('Что-то пошло не так',err)

input('Для выхода нажмите Enter.')

Введите дату в формате ГГГГ-ММ-ДД или нажмите Enter для текущего дня:

Читаю новости за 2022-06-14 14:15:58.480808
Читаю новости tomsk123...
Прочитано.
Читаю новости ghest_tomsk...
Прочитано.
Читаю новости ya_tomich...
Прочитано.
Читаю новости tomskie...
Прочитано.
Читаю новости typical_tomsk...
Прочитано.
Читаю новости tomskvkurse...
Прочитано.
Читаю новости inctsk...
Прочитано.
Читаю новости znaigorod...
Прочитано.
Читаю новости goodcity70...
Прочитано.
Читаю новости black_list_tomsk...
Прочитано.
Читаю новости dortom...
Прочитано.
Записываю файл excel...
Готово, можете пользоваться!
Для выхода нажмите Enter.


''